In [30]:
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import pinecone

In [31]:
# Extracting Data from PDF
def load_pdf(data):
  loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
  documents = loader.load()
  return documents
extracted_data = load_pdf("data/")

FileNotFoundError: Directory not found: 'data/'

In [ ]:
# Creating Text Chunks
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks
text_chunks = text_split(extracted_data)
print("Length of the chunks: ", len(text_chunks)) # 7020

Length of the chunks:  5860


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embeddings


In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
# Query Embedding:
query_result = embeddings.embed_query("Hello World")
print("Length: ", len(query_result)) # 384

Length:  384


UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '0ba176d8b4a9f09ba2624687690c8b15', 'Date': 'Mon, 17 Feb 2025 16:50:52 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [ ]:
import os

# If you're using an environment variable for the API key
os.environ["PINECONE_API_KEY"] = "pcsk_5ocNt6_6E4Gy6T3EUt8aSyv2fVBXLpTXyDAJXsfQjbMx3Tp2WKYkGuUPDpAr1NSQDNyQDM"

# Or directly in the code (not recommended for production)
api_key = "pcsk_5ocNt6_6E4Gy6T3EUt8aSyv2fVBXLpTXyDAJXsfQjbMx3Tp2WKYkGuUPDpAr1NSQDNyQDM"
pc = Pinecone(api_key=api_key)


In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
    PINCONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [ ]:
# Creating a Pinecone Cluster
# - Login into Pinecone: https://www.pinecone.io/ > Dashboard
# - API Keys > Create API Key and use it
# - Indexes > Create Index: medical-chatbot, 384 Dimensions, Cosine (HuggingFace Model: sentence-transformers/all-MiniLM-L6-v2) > Use the ENVIRONMENT

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)
index_name="medical-chatbot"

In [ ]:
# Creating embeddings for each of the text chunks and storing it
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
# - Goto the Pinecone website, and view all the vectors that are stored. Total: 7020
# - Now, performing semantic index: Doing Semantic or Similarity Search
# If we already have an index we can load it like this
docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "What are Allergies?"
docs = docsearch.similarity_search(query, k=3)
print("Result", docs)

In [ ]:
# Implementing the PromptTemplate
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else...
Helpful answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin", model_type="llama", config={"max_new_tokens":512, "temperature":0.8})

In [ ]:
# Performing Retrieval Question-Answering
qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k':2}), return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

while True:
  user_input = input(f"Input Prompt: ")
  result = qa({"query": user_input})
  print("Response: ", result["result"])

In [32]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vector_store = FAISS.from_documents(text_chunks, embeddings)

# Convert FAISS into a retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 2})


NameError: name 'text_chunks' is not defined

In [33]:
import os
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# ================================
# Step 1: Load PDF Documents
# ================================
def load_pdf(data_path):
    """Load all PDF documents from the specified directory."""
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf("data/")  # Ensure "data/" contains PDFs

# ================================
# Step 2: Split Text into Chunks
# ================================
def text_split(extracted_data):
    """Split the extracted text into smaller chunks."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)

print(f"✅ Successfully split {len(text_chunks)} chunks!")

# ================================
# Step 3: Initialize FAISS Vector Store
# ================================
# Load Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Ensure text_chunks is properly initialized before using FAISS
if len(text_chunks) > 0:
    vector_store = FAISS.from_documents(text_chunks, embeddings)
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    print("✅ FAISS Vector Store Created Successfully!")
else:
    print("⚠️ No text chunks found. Check if your PDFs are loading correctly.")


FileNotFoundError: Directory not found: 'data/'

In [ ]:
%pwd
import os
os.chdir("")
%pwd

FileNotFoundError: [WinError 2] The system cannot find the file specified: '\\medical-chatbot.ipynb'

In [39]:
%pwd

'd:\\'

In [40]:
import os
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# ================================
# Step 1: Load PDF Documents
# ================================
def load_pdf():
    """Load all PDF documents from the 'data/' directory."""
    data_path = os.path.join(os.getcwd(), "data")  # Get full path dynamically
    if not os.path.exists(data_path):
        print(f"⚠️ The directory '{data_path}' does not exist. Please check the path.")
        return []

    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf()

# ================================
# Step 2: Split Text into Chunks
# ================================
def text_split(extracted_data):
    """Split the extracted text into smaller chunks."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)

if not text_chunks:
    print("⚠️ No text chunks found. Check if your PDFs are loading correctly.")
else:
    print(f"✅ Successfully split {len(text_chunks)} chunks!")

# ================================
# Step 3: Initialize FAISS Vector Store
# ================================
# Load Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if text_chunks:
    vector_store = FAISS.from_documents(text_chunks, embeddings)
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    print("✅ FAISS Vector Store Created Successfully!")
else:
    print("⚠️ Cannot create FAISS vector store. No text chunks available.")


⚠️ The directory 'd:\data' does not exist. Please check the path.
⚠️ No text chunks found. Check if your PDFs are loading correctly.
⚠️ Cannot create FAISS vector store. No text chunks available.


In [41]:
os.getcwd()

'd:\\'

In [42]:
import os
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# ================================
# Step 1: Load PDF Documents (Using Absolute Path)
# ================================
def load_pdf():
    """Load all PDF documents from the 'D:\MEDICAL_CHATBOT\data\' directory."""
    
    project_path = r"D:\Medical-Chatbot-Using-Llama2-main"  # Set your project folder path
    data_path = os.path.join(project_path, "data")  # Construct the full data folder path

    if not os.path.exists(data_path):
        print(f"⚠️ The directory '{data_path}' does not exist. Please check the path.")
        return []

    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf()

# ================================
# Step 2: Split Text into Chunks
# ================================
def text_split(extracted_data):
    """Split the extracted text into smaller chunks."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)

if not text_chunks:
    print("⚠️ No text chunks found. Check if your PDFs are loading correctly.")
else:
    print(f"✅ Successfully split {len(text_chunks)} chunks!")

# ================================
# Step 3: Initialize FAISS Vector Store
# ================================
# Load Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if text_chunks:
    vector_store = FAISS.from_documents(text_chunks, embeddings)
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    print("✅ FAISS Vector Store Created Successfully!")
else:
    print("⚠️ Cannot create FAISS vector store. No text chunks available.")


✅ Successfully split 5860 chunks!


ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).